In [1]:
import pandas as pd

In [ ]:
if not 'ROOT' in globals():
    import os
    ROOT = os.getcwd() + '/../..'

In [2]:
# Make sure output dir exists
#if not os.path.exists(outdir):
#    os.mkdir(outdir)

In [3]:
import os

class ECDC:
    
    indir = ROOT + '/data/raw/ecdc/'
    outdir = ROOT + '/data/processed/ecdc/'

    data = None;
    population2018 = None
    
    def load():
        ECDC.data = pd.read_csv(
            ECDC.indir + 'ecdc.csv',
            names = [ 'dateRep', 'day', 'month', 'year', 'cases', 'deaths', 'name', 'code2', 'code3', 'population2018', 'continent' ],
            header=0,
            #index_col=['date', 'code3'],
            dtype = {
                'name': 'category',
                'code2': 'category',
                'code3': 'category',
                'continent': 'category'
            },
            parse_dates = {
                'date': ['year', 'month', 'day'],
                'reported': ['dateRep']
            }, keep_date_col=False
        )

        ECDC.population2018 = ECDC.data[['code3', 'population2018']].drop_duplicates()
        ECDC.population2018.columns = ['country', 'population']
        ECDC.population2018.set_index('country', inplace=True)
        
        
        
        ECDC.data.drop(['population2018'], axis=1, inplace=True)    
            
        # Following countries are missing 2 char country codes (= lost when dropping NaN):
        # - Namibia

        ECDC.data = ECDC.data[ ECDC.data['code2'].notnull()]

        # Following countries are missing 3 char country codes (= lost when dropping NaN):
        # - Anguilla
        # - Bonaire, Saint Eustatius and Saba
        # - Cases_on_an_international_conveyance_Japan
        # - Czechia
        # - Falkland_Islands_(Malvinas)

        ECDC.data = ECDC.data[ ECDC.data['code3'].notnull()]
        
        
        ECDC.data.drop(['code2'], axis=1, inplace=True)
        ECDC.data.rename(columns={'code3': 'country'}, inplace=True)
        
    
        ECDC.data.set_index(['date', 'country'], inplace=True)
        
        
    def cases_and_deaths():
        return ECDC.data.drop(['name', 'reported', 'continent' ], axis=1)

    def cases():
        return ECDC.cases_and_deaths()['cases']

    def deaths():
        return ECDC.cases_and_deaths()['deaths']

    def countries():
        return ECDC.data.reset_index()[['country', 'name']].set_index('country').drop_duplicates()
    
    def population():
        return ECDC.population2018

In [4]:
ECDC.load()

In [5]:
if False:
    ECDC.cases_and_deaths()['cases'].unstack().to_csv(
        outdir + 'cases.csv',
        sep='|'
    )

    ECDC.cases_and_deaths()['deaths'].unstack().to_csv(
        outdir + 'deaths.csv',
        sep='|'
    )

    #ECDC.cases_and_deaths()['cases'].unstack()['DNK'].to_csv(
    #    'python/cases-DNK.csv',
    #    sep='|'
    #)

In [6]:
if False:
    ECDC.countries().to_csv(
        outdir + 'countries.csv',
        sep='|'
    )